In [153]:
import os
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
from torch.nn import functional as F
import pandas as pd

### My AutoGradient

In [154]:
#Input

batch_size = 1
hidden_node = 21 # 2d+1 
input_data = 10 #d=10
output_data = 1
hidden_layer = 10
learning_rate = 1e-6


In [155]:
#define x, y
runif = torch.distributions.Uniform(0,1)
x = runif.sample((batch_size,input_data))
y = x.pow(2).sum()/2

#Weight & Bias
w1 = torch.randn(input_data, hidden_node) #1
b1 = torch.randn(1, hidden_node)
w_list=[w1]
b_list=[b1]
for i in range(9):
    w_list.append(torch.randn(hidden_node, hidden_node)) #1+9
    b_list.append(torch.randn(1, hidden_node))
w11 = torch.randn(hidden_node, output_data) 
b11 = torch.randn(1, output_data)
w_list.append(w11)
b_list.append(b11)


#Forward-propagation
h_list=[F.relu(torch.matmul(x, w_list[0])+b_list[0],)]#1 0
for i in range(9):
    h_list.append(F.relu(torch.matmul(h_list[i], w_list[i+1])+b_list[i+1])) #9 0-8
y_pred = F.relu(torch.matmul(h_list[9], w_list[10])+b_list[10])

#Loss Function
loss = (y_pred-y).pow(2).sum()

#Back-propagation
grad_y_pred = 2*(y_pred - y)
w_grad = [0]*11
b_grad = [0]*11

for i in range(11):
    if i == 0:
        b_grad[10 - i] = grad_y_pred
        w_grad[10 - i] = (b_grad[10-i] * h_list[9-i].T )
    elif i == 10:
        b_grad[10 - i] = torch.mm(b_grad[11-i],w_list[11 - i].T)
        b_grad[10 - i][h_list[10 - i]<=0] = 0
        w_grad[10 - i] = torch.mm(x.T,b_grad[10-i])
    else:
        b_grad[10 - i] = torch.mm(b_grad[11-i],w_list[11 - i].T)
        b_grad[10 - i][h_list[10 - i]<=0] = 0
        w_grad[10 - i] = torch.mm(h_list[9-i].T,b_grad[10-i])

In [156]:
f = open('my_grad_b.txt','w')
for i in range(len(b_grad)):
    for j in range(len(b_grad[i])):
        f.write(str(b_grad[i][j]))
        f.write('\t')
        f.write('\n')
f.close()

In [157]:
f = open('my_grad_w.txt','w')
for i in range(len(w_grad)):
        f.write(str(w_grad[i]))
        f.write('\t')
        f.write('\n')
f.close()

### Torch Gradient

In [158]:
# weight & Bias
w1_auto = torch.tensor(w1.detach().cpu().numpy(),requires_grad=True)  #1
b1_auto = torch.tensor(b1.detach().cpu().numpy(),requires_grad=True) 
w_auto_list=[w1_auto]
b_auto_list=[b1_auto]
for i in range(9):
    w_auto_list.append(torch.tensor(w_list[i+1].detach().cpu().numpy(),requires_grad=True)) #1+9
    b_auto_list.append(torch.tensor(b_list[i+1].detach().cpu().numpy(),requires_grad=True))
w_auto_11 = torch.tensor(w11.detach().cpu().numpy(),requires_grad=True)
b_auto_11 = torch.tensor(b11.detach().cpu().numpy(),requires_grad=True) 
w_auto_list.append(w_auto_11)
b_auto_list.append(b_auto_11)

#Auto Forward-propagation
h_auto_list=[F.relu(torch.matmul(x, w_auto_list[0])+b_auto_list[0])]#1 0

for i in range(9):
    h_auto_list.append(F.relu(torch.matmul(h_auto_list[i], w_auto_list[i+1])+b_auto_list[i+1])) #9 0-8
y_auto_pred = F.relu(torch.matmul(h_auto_list[9], w_auto_11)+b_auto_11).sum()

#Loss Function
loss = (y_auto_pred-y).pow(2).sum()
#back-propagation
loss.backward()

In [160]:
f = open('torch_grad_b.txt','w')
for i in range(len(b_auto_list)):
    f.write(str(b_auto_list[i].grad))
    f.write('\t')
    f.write('\n')
f.close()

In [161]:
f = open('torch_grad_w.txt','w')
for i in range(len(w_auto_list)):
    f.write(str(w_auto_list[i].grad))
    f.write('\t')
    f.write('\n')
f.close()